### RLOSS Factor:

- R_LOSS = Input Image vs Output Image
- KL_LOSS = Distribution concluded from Input Image vs. Standard Distribution

``
LOSS = R_LOSS_FACTOR * R_LOSS + KL_LOSS
``

Hence, 
- Low R_LOSS_FACTOR -> high KL_LOSS weighing:
 - output images try to be similar to all training images (blurry)


- High R_LOSS_FACTOR -> low KL_LOSS weighing:
 - output images are similar to input images and have less variation leeway


In [1]:
#must be very first statement

import tensorflow as tf
tf.debugging.set_log_device_placement(True)
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
if tf.test.gpu_device_name():
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))
else:
    print("Please install GPU version of TF")

Num GPUs Available:  1
Default GPU Device: /device:GPU:0


In [2]:
#%config IPCompleter.greedy=True

In [3]:
#utils'

import numpy as np
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from keras.datasets import fashion_mnist
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.callbacks import Callback, LearningRateScheduler
import numpy as np
import matplotlib.pyplot as plt
import os
from keras.datasets import mnist
import PIL


def getDigits(show=False):
    (x_train, t_train), (x_test, t_test) = mnist.load_data()

    #print(x_train.shape)

    if show:
        showImages(x_train, t_train,5)

    return x_train,t_train,x_test, t_test


#### CALLBACKS (https://github.com/davidADSP/GDL_code/blob/master/utils/callbacks.py)
class CustomCallback(Callback):

    def __init__(self, run_folder, print_every_n_batches, initial_epoch, vae):
        self.epoch = initial_epoch
        self.run_folder = run_folder
        self.print_every_n_batches = print_every_n_batches
        self.vae = vae

    def on_batch_end(self, batch, logs={}):
        if batch % self.print_every_n_batches == 0:
            z_new = np.random.normal(size=(1, self.vae.z_dim))
            reconst = self.vae.decoder.predict(np.array(z_new))[0].squeeze()

            filepath = os.path.join(self.run_folder, 'images',
                                    'img_' + str(self.epoch).zfill(3) + '_' + str(batch) + '.jpg')
            if len(reconst.shape) == 2:
                plt.imsave(filepath, reconst, cmap='gray_r')
            else:
                plt.imsave(filepath, reconst)

    def on_epoch_begin(self, epoch, logs={}):
        self.epoch += 1

#### CALLBACKS (https://github.com/davidADSP/GDL_code/blob/master/utils/callbacks.py)
def step_decay_schedule(initial_lr, decay_factor=0.5, step_size=1):
    '''
    Wrapper function to create a LearningRateScheduler with step decay schedule.
    '''

    def schedule(epoch):
        new_lr = initial_lr * (decay_factor ** np.floor(epoch / step_size))

        return new_lr

    return LearningRateScheduler(schedule)

Using TensorFlow backend.


In [4]:
#data prep

# changes:
#
# - keras load cifar 10 inst. getDigits (MNIST) from utils
#
# - allow selection of training subset based on label
# - for softmax classification new label order: 2,3,8 -> 0,1,2
#
# - reshape:
#   x_train=x_train.reshape(x_train.shape[0],32,32,3)
#   x_test=x_test.reshape(x_test.shape[0],32,32,3)
#   (instead of 28,28,1)
#
# Aktivierung:
# My data was not best approximated by a Bernoulli but a Gaussian. 
# So, using a GaussianReconstructionDistribution with a TANH gave better results.
# The DL4J JavaDocs state: 
# "For activation functions, identity and perhaps tanh are typical - though tanh (unlike identity) 
# implies a minimum/maximum possible value for mean and log variance. Asymmetric activation functions 
# such as sigmoid or relu should be avoided". 

from keras.datasets import cifar10
cifar10.load_data()

my_labels = [1]
all_label_names = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
label_names = []
for label_index in my_labels:
    label_names.append(all_label_names[label_index])  

print("Defining new Labeling:")
print(dict(zip(range(len(my_labels)),label_names)))

#if my_labels = [5,6,8] then 5 returns 0, 6 returns 1, 8 returns 2, ...
def convert_label(label):
    return dict(zip(my_labels,range(len(my_labels))))[label]

def label_name(num):
    return label_names[num]

# The data, split between train and test sets:
(x_train_all, y_train_all), (x_test_all, y_test_all) = cifar10.load_data()
    
#temp lists
x_train = []
y_train_numerical = []

#filter training data for my_labels
for i in range(len(x_train_all)):
    if y_train_all[i] in my_labels:
        x_train.append(x_train_all[i])
        y_train_numerical.append(convert_label(y_train_all[i][0]))
    else:
        pass

x_train = np.array(x_train)
y_train_numerical = np.array(y_train_numerical)

print("Training Data:\n")
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'samples,', y_train_numerical.shape[0], 'labels')
print("\nClass  |  Counts:")
(unique, counts) = np.unique(y_train_numerical, return_counts=True)
for i, label in enumerate(unique):
    print(label_name(label),"\t", counts[i])


x_test = []
y_test_numerical = []

#filter test data
for i in range(len(x_test_all)):
    if y_test_all[i] in my_labels:
        x_test.append(x_test_all[i])
        y_test_numerical.append(convert_label(y_test_all[i][0]))
    else:
        pass

x_test = np.array(x_test)
y_test_numerical = np.array(y_test_numerical)

print("\n\nTesting Data:\n")
print('x_test shape:', x_test.shape)
print(x_test.shape[0], 'samples,', y_test_numerical.shape[0], 'labels')
print("\nClass  |  Counts:")
(unique, counts) = np.unique(y_test_numerical, return_counts=True)
for i, label in enumerate(unique):
    print(label_name(label),"\t", counts[i])
    
x_train=x_train.reshape(x_train.shape[0],32,32,3)
x_test=x_test.reshape(x_test.shape[0],32,32,3)

x_train = x_train.astype('float32')[:500]
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255


Defining new Labeling:
{0: 'automobile'}
Training Data:

x_train shape: (5000, 32, 32, 3)
5000 samples, 5000 labels

Class  |  Counts:
automobile 	 5000


Testing Data:

x_test shape: (1000, 32, 32, 3)
1000 samples, 1000 labels

Class  |  Counts:
automobile 	 1000


In [5]:
#model class
from keras.layers import Input, Conv2D, Flatten, Dense, Conv2DTranspose, Reshape, Lambda, Activation, BatchNormalization, LeakyReLU, Dropout
from keras.models import Model
from keras import backend as K
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint
from keras.utils import plot_model
from keras import callbacks

import numpy as np
import json
import os
import pickle

# Verteilung + KL:
# https://stats.stackexchange.com/questions/402569/why-do-we-use-gaussian-distributions-in-variational-autoencoder
#
# TODO:
# sigmoid vs tanh in decoder activation
# batchnormalization: ON
# (multivariate distribution instead of gaussian normal)

class VariationalAutoencoder():
    def __init__(self
                 , input_dim
                 , encoder_conv_filters
                 , encoder_conv_kernel_size
                 , encoder_conv_strides
                 , decoder_conv_t_filters
                 , decoder_conv_t_kernel_size
                 , decoder_conv_t_strides
                 , z_dim
                 , use_batch_norm=False
                 , use_dropout=False
                 ):

        self.name = 'variational_autoencoder'

        self.input_dim = input_dim
        self.encoder_conv_filters = encoder_conv_filters
        self.encoder_conv_kernel_size = encoder_conv_kernel_size
        self.encoder_conv_strides = encoder_conv_strides
        self.decoder_conv_t_filters = decoder_conv_t_filters
        self.decoder_conv_t_kernel_size = decoder_conv_t_kernel_size
        self.decoder_conv_t_strides = decoder_conv_t_strides
        self.z_dim = z_dim

        self.use_batch_norm = use_batch_norm
        self.use_dropout = use_dropout

        self.n_layers_encoder = len(encoder_conv_filters)
        self.n_layers_decoder = len(decoder_conv_t_filters)

        self._build()

    def _build(self):

        ### THE ENCODER
        encoder_input = Input(shape=self.input_dim, name='encoder_input')

        x = encoder_input

        for i in range(self.n_layers_encoder):
            conv_layer = Conv2D(
                filters=self.encoder_conv_filters[i]
                , kernel_size=self.encoder_conv_kernel_size[i]
                , strides=self.encoder_conv_strides[i]
                , padding='same'
                , name='encoder_conv_' + str(i)
            )

            x = conv_layer(x)

            if self.use_batch_norm:
                x = BatchNormalization()(x)

            x = LeakyReLU()(x) # for encoding

            if self.use_dropout:
                x = Dropout(rate=0.25)(x)

        shape_before_flattening = K.int_shape(x)[1:]
#-----------------------------
        print("shape_bef_flat",shape_before_flattening)
        x = Flatten()(x)
        print("shape_aft_flat",x)
        self.mu = Dense(self.z_dim, name='mu')(x)
        self.log_var = Dense(self.z_dim, name='log_var')(x)

        self.encoder_mu_log_var = Model(encoder_input, (self.mu, self.log_var))

        def sampling(args):
            mu, log_var = args
            epsilon = K.random_normal(shape=K.shape(mu), mean=0., stddev=1) #gaussian standardnormalvert.
            return mu + K.exp(log_var / 2) * epsilon #try without /2
    
        encoder_output = Lambda(sampling, name='encoder_output')([self.mu, self.log_var])
        print("encoder_output: ",encoder_output)
        self.encoder = Model(encoder_input, encoder_output)

        ### THE DECODER

        decoder_input = Input(shape=(self.z_dim,), name='decoder_input')
        print("dec_input",decoder_input)
        x = Dense(np.prod(shape_before_flattening))(decoder_input)
        x = Reshape(shape_before_flattening)(x)

        for i in range(self.n_layers_decoder):
            conv_t_layer = Conv2DTranspose(
                filters=self.decoder_conv_t_filters[i]
                , kernel_size=self.decoder_conv_t_kernel_size[i]
                , strides=self.decoder_conv_t_strides[i]
                , padding='same'
                , name='decoder_conv_t_' + str(i)
            )

            x = conv_t_layer(x)

            if i < self.n_layers_decoder - 1:
                if self.use_batch_norm:
                    x = BatchNormalization()(x)
                x = LeakyReLU()(x)
                if self.use_dropout:
                    x = Dropout(rate=0.25)(x)
            else:
                x = Activation('sigmoid')(x)

        decoder_output = x
        self.decoder = Model(decoder_input, decoder_output)

        ### THE FULL VAE
        model_input = encoder_input
        model_output = self.decoder(encoder_output)
        
        self.model = Model(model_input, model_output)

    def compile(self, learning_rate, r_loss_factor):
        self.learning_rate = learning_rate

        ### COMPILATION
        def vae_r_loss(y_true, y_pred):
            r_loss = K.mean(K.square(y_true - y_pred), axis=[1, 2, 3])
            return r_loss * r_loss_factor

        def vae_kl_loss(y_true, y_pred):
            kl_loss = -0.5 * K.sum(1 + self.log_var - K.square(self.mu) - K.exp(self.log_var), axis=1)
            return kl_loss

        def vae_loss(y_true, y_pred):
            r_loss = vae_r_loss(y_true, y_pred)
            kl_loss = vae_kl_loss(y_true, y_pred)
            return r_loss + kl_loss

        optimizer = Adam(lr=learning_rate)
        self.model.compile(optimizer=optimizer, loss=vae_loss, metrics=[vae_r_loss, vae_kl_loss])

    def save(self, folder="run"):
        if not os.path.exists(folder):
            os.makedirs(folder)
            os.makedirs(os.path.join(folder, 'viz'))
            os.makedirs(os.path.join(folder, 'weights'))
            os.makedirs(os.path.join(folder, 'images'))

        with open(os.path.join(folder, 'params.pkl'), 'wb') as f:
            pickle.dump([
                self.input_dim
                , self.encoder_conv_filters
                , self.encoder_conv_kernel_size
                , self.encoder_conv_strides
                , self.decoder_conv_t_filters
                , self.decoder_conv_t_kernel_size
                , self.decoder_conv_t_strides
                , self.z_dim
                , self.use_batch_norm
                , self.use_dropout
            ], f)

        self.plot_model(folder)

    def load_weights(self, filepath):
        self.model.load_weights(filepath)

    def train(self, x_train, batch_size, epochs, run_folder, print_every_n_batches=100, initial_epoch=0, lr_decay=1):

        custom_callback = CustomCallback(run_folder, print_every_n_batches, initial_epoch, self)
        lr_sched = step_decay_schedule(initial_lr=self.learning_rate, decay_factor=lr_decay, step_size=1)

        checkpoint_filepath = os.path.join(run_folder, "weights/weights.h5")
        
        model_checkpoint_callback = callbacks.ModelCheckpoint(
            filepath=checkpoint_filepath,
            save_weights_only=True,
            monitor='val_acc',
            mode='max')

        
        #checkpoint1 = ModelCheckpoint(checkpoint_filepath, save_weights_only=True, verbose=1)
        #checkpoint2 = ModelCheckpoint(os.path.join(run_folder, 'weights/weights.h5'), save_weights_only=True, verbose=1)

        callbacks_list = [custom_callback, lr_sched, model_checkpoint_callback]

        self.model.fit(
            x_train
            , x_train
            , batch_size=batch_size
            , shuffle=True
            , epochs=epochs
            , initial_epoch=initial_epoch
            , callbacks=callbacks_list
        )

    def plot_model(self, run_folder):
        plot_model(self.model, to_file=os.path.join(run_folder, 'viz/model.png'), show_shapes=True,
                   show_layer_names=True)
        plot_model(self.encoder, to_file=os.path.join(run_folder, 'viz/encoder.png'), show_shapes=True,
                   show_layer_names=True)
        plot_model(self.decoder, to_file=os.path.join(run_folder, 'viz/decoder.png'), show_shapes=True,
                   show_layer_names=True)



In [6]:
#training
SECTION = 'vae'
RUN_ID = '0002'
DATA_NAME = 'digits'
RUN_FOLDER = 'run_losscompare'

cwd = os.getcwd()
print(cwd)
if not os.path.exists(RUN_FOLDER):
    os.mkdir(RUN_FOLDER)
    os.mkdir(os.path.join(RUN_FOLDER, 'viz'))
    os.mkdir(os.path.join(RUN_FOLDER, 'images'))
    os.mkdir(os.path.join(RUN_FOLDER, 'weights'))

mode =  'build' #'load' #




S:\Hub\ML_SS_2020


In [ ]:

# größeres modell
#Wichtig: Beim Output Filteranzahl = 3 -> Dreidimensionales Ergebnis für RGB Interpretation
vae = VariationalAutoencoder(
    input_dim = (32,32,3)
    , encoder_conv_filters = [128,128,128,512]
    , encoder_conv_kernel_size = [2,2,3,4]
    , encoder_conv_strides = [1,1,2,2]
    , decoder_conv_t_filters = [512,128,128,3]
    , decoder_conv_t_kernel_size = [4,3,2,2]
    , decoder_conv_t_strides = [2,2,1,1]
    , z_dim = 128
)
vae.save(RUN_FOLDER)

vae.encoder.summary()

LEARNING_RATE = 0.0001
R_LOSS_FACTOR = 100
vae.compile(LEARNING_RATE, R_LOSS_FACTOR)

BATCH_SIZE = 32
EPOCHS = 500
PRINT_EVERY_N_BATCHES = 100
INITIAL_EPOCH = 0

vae.train(
    x_train
    , batch_size = BATCH_SIZE
    , epochs = EPOCHS
    , run_folder = RUN_FOLDER
    , print_every_n_batches = PRINT_EVERY_N_BATCHES
    , initial_epoch = INITIAL_EPOCH
)
vae.save()


In [ ]:
# größeres modell
#Wichtig: Beim Output Filteranzahl = 3 -> Dreidimensionales Ergebnis für RGB Interpretation
vae = VariationalAutoencoder(
    input_dim = (32,32,3)
    , encoder_conv_filters = [128,128,128,512]
    , encoder_conv_kernel_size = [2,2,3,4]
    , encoder_conv_strides = [1,1,2,2]
    , decoder_conv_t_filters = [512,128,128,3]
    , decoder_conv_t_kernel_size = [4,3,2,2]
    , decoder_conv_t_strides = [2,2,1,1]
    , z_dim = 2
)
vae.save(RUN_FOLDER)

vae.encoder.summary()

LEARNING_RATE = 0.0001
R_LOSS_FACTOR = 5000
vae.compile(LEARNING_RATE, R_LOSS_FACTOR)

BATCH_SIZE = 32
EPOCHS = 1000
PRINT_EVERY_N_BATCHES = 100
INITIAL_EPOCH = 0

vae.train(
    x_train
    , batch_size = BATCH_SIZE
    , epochs = EPOCHS
    , run_folder = RUN_FOLDER
    , print_every_n_batches = PRINT_EVERY_N_BATCHES
    , initial_epoch = INITIAL_EPOCH
)
vae.save()


Executing op RandomUniform in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op RandomUniform in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op RandomUniform in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op RandomUniform in device /job:localhost/replica:0/task:0/device:GPU:0
shape_bef_flat (8, 8, 512)
shape_aft_flat Tensor("flatten_4/Reshape:0", shape=(None, None), dtype=float32)
Executing op RandomUniform in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op RandomUniform in device /job:localhost/replica:0/task:0/device:GPU:0
encoder_output:  Tensor("encoder_output_3/add:0", shape=(None, 2), dtype=float32)
dec_input Tensor("decoder_input_3:0", shape=(None, 2), dtype=float32)
Executing op RandomUniform in device /job:localhost/replica:0/task:0/device:GPU:0
Executi

C:\Users\Admin\Anaconda3\envs\ml20\lib\site-packages\keras\callbacks\callbacks.py:95: RuntimeWarning: Method (on_train_batch_end) is slow compared to the batch update (0.485000). Check your callbacks.
  % (hook_name, delta_t_median), RuntimeWarning)


500/500 [==============================] - 9s 19ms/step - loss: 379.5561 - vae_r_loss: 379.5302 - vae_kl_loss: 0.0259
Epoch 2/1000
500/500 [==============================] - 1s 2ms/step - loss: 370.0878 - vae_r_loss: 369.8786 - vae_kl_loss: 0.2093
Epoch 3/1000
500/500 [==============================] - 1s 2ms/step - loss: 349.3964 - vae_r_loss: 348.2130 - vae_kl_loss: 1.1835
Epoch 4/1000
500/500 [==============================] - 1s 2ms/step - loss: 318.6367 - vae_r_loss: 312.4237 - vae_kl_loss: 6.2130
Epoch 5/1000
500/500 [==============================] - 1s 2ms/step - loss: 298.1541 - vae_r_loss: 293.8375 - vae_kl_loss: 4.3165
Epoch 6/1000
500/500 [==============================] - 1s 2ms/step - loss: 278.0506 - vae_r_loss: 272.5572 - vae_kl_loss: 5.4934
Epoch 7/1000
500/500 [==============================] - 1s 2ms/step - loss: 261.8338 - vae_r_loss: 256.0896 - vae_kl_loss: 5.7442
Epoch 8/1000
500/500 [==============================] - 1s 2ms/step - loss: 251.1158 - vae_r_loss: 245

500/500 [==============================] - 1s 2ms/step - loss: 224.4668 - vae_r_loss: 219.5621 - vae_kl_loss: 4.9046
Epoch 64/1000
500/500 [==============================] - 1s 2ms/step - loss: 224.5367 - vae_r_loss: 219.6679 - vae_kl_loss: 4.8687
Epoch 65/1000
500/500 [==============================] - 1s 2ms/step - loss: 224.3741 - vae_r_loss: 219.2871 - vae_kl_loss: 5.0870
Epoch 66/1000
500/500 [==============================] - 1s 2ms/step - loss: 224.5665 - vae_r_loss: 219.5808 - vae_kl_loss: 4.9856
Epoch 67/1000
500/500 [==============================] - 1s 2ms/step - loss: 224.1684 - vae_r_loss: 219.0671 - vae_kl_loss: 5.1014
Epoch 68/1000
500/500 [==============================] - 1s 2ms/step - loss: 224.3047 - vae_r_loss: 219.4182 - vae_kl_loss: 4.8865
Epoch 69/1000
500/500 [==============================] - 1s 2ms/step - loss: 224.6350 - vae_r_loss: 219.6636 - vae_kl_loss: 4.9713
Epoch 70/1000
500/500 [==============================] - 1s 2ms/step - loss: 224.1327 - vae_r_los

500/500 [==============================] - 1s 2ms/step - loss: 219.1977 - vae_r_loss: 214.0470 - vae_kl_loss: 5.1508
Epoch 126/1000
500/500 [==============================] - 1s 2ms/step - loss: 219.1676 - vae_r_loss: 214.0729 - vae_kl_loss: 5.0947
Epoch 127/1000
500/500 [==============================] - 1s 2ms/step - loss: 219.3608 - vae_r_loss: 214.0870 - vae_kl_loss: 5.2738
Epoch 128/1000
500/500 [==============================] - 1s 2ms/step - loss: 219.2264 - vae_r_loss: 213.9649 - vae_kl_loss: 5.2615
Epoch 129/1000
500/500 [==============================] - 1s 2ms/step - loss: 219.8452 - vae_r_loss: 214.6522 - vae_kl_loss: 5.1930
Epoch 130/1000
500/500 [==============================] - 1s 2ms/step - loss: 219.2096 - vae_r_loss: 213.8973 - vae_kl_loss: 5.3123
Epoch 131/1000
500/500 [==============================] - 1s 2ms/step - loss: 218.7887 - vae_r_loss: 213.5628 - vae_kl_loss: 5.2259
Epoch 132/1000
500/500 [==============================] - 1s 2ms/step - loss: 219.0831 - va

500/500 [==============================] - 1s 2ms/step - loss: 212.2603 - vae_r_loss: 206.3686 - vae_kl_loss: 5.8917
Epoch 187/1000
500/500 [==============================] - 1s 2ms/step - loss: 211.0264 - vae_r_loss: 205.4377 - vae_kl_loss: 5.5888
Epoch 188/1000
500/500 [==============================] - 1s 2ms/step - loss: 211.1559 - vae_r_loss: 205.4827 - vae_kl_loss: 5.6732
Epoch 189/1000
500/500 [==============================] - 1s 2ms/step - loss: 211.6618 - vae_r_loss: 205.9601 - vae_kl_loss: 5.7017
Epoch 190/1000
500/500 [==============================] - 1s 2ms/step - loss: 210.7362 - vae_r_loss: 204.8612 - vae_kl_loss: 5.8750
Epoch 191/1000
500/500 [==============================] - 1s 2ms/step - loss: 210.4700 - vae_r_loss: 204.6221 - vae_kl_loss: 5.8480
Epoch 192/1000
500/500 [==============================] - 1s 2ms/step - loss: 211.3314 - vae_r_loss: 205.6613 - vae_kl_loss: 5.6700
Epoch 193/1000
500/500 [==============================] - 1s 2ms/step - loss: 210.8715 - va

500/500 [==============================] - 1s 2ms/step - loss: 204.8529 - vae_r_loss: 198.4875 - vae_kl_loss: 6.3653
Epoch 247/1000
500/500 [==============================] - 1s 2ms/step - loss: 204.7062 - vae_r_loss: 198.5869 - vae_kl_loss: 6.1193
Epoch 248/1000
500/500 [==============================] - 1s 2ms/step - loss: 206.5770 - vae_r_loss: 200.3706 - vae_kl_loss: 6.2064
Epoch 249/1000
500/500 [==============================] - 1s 2ms/step - loss: 206.3600 - vae_r_loss: 200.2064 - vae_kl_loss: 6.1536
Epoch 250/1000
500/500 [==============================] - 1s 2ms/step - loss: 205.9253 - vae_r_loss: 199.7313 - vae_kl_loss: 6.1940
Epoch 251/1000
500/500 [==============================] - 1s 2ms/step - loss: 204.5398 - vae_r_loss: 198.4296 - vae_kl_loss: 6.1103
Epoch 252/1000
500/500 [==============================] - 1s 2ms/step - loss: 205.0155 - vae_r_loss: 198.8916 - vae_kl_loss: 6.1240
Epoch 253/1000
500/500 [==============================] - 1s 2ms/step - loss: 204.5779 - va

500/500 [==============================] - 1s 2ms/step - loss: 200.8273 - vae_r_loss: 194.2134 - vae_kl_loss: 6.6140
Epoch 308/1000
500/500 [==============================] - 1s 2ms/step - loss: 200.4350 - vae_r_loss: 193.9864 - vae_kl_loss: 6.4486
Epoch 309/1000
500/500 [==============================] - 1s 2ms/step - loss: 200.1998 - vae_r_loss: 193.5292 - vae_kl_loss: 6.6706
Epoch 310/1000
416/500 [=======================>......] - ETA: 0s - loss: 199.1099 - vae_r_loss: 192.5713 - vae_kl_loss: 6.5386

In [ ]:
#Higher Z Dim and Lower R Loss Factor

vae = VariationalAutoencoder(
    input_dim = (32,32,3)
    , encoder_conv_filters = [128,128,128,512]
    , encoder_conv_kernel_size = [2,2,3,4]
    , encoder_conv_strides = [1,1,2,2]
    , decoder_conv_t_filters = [512,128,128,3]
    , decoder_conv_t_kernel_size = [4,3,2,2]
    , decoder_conv_t_strides = [2,2,1,1]
    , z_dim = 400
)
vae.save(RUN_FOLDER)

LEARNING_RATE = 0.0005
R_LOSS_FACTOR = 250
vae.compile(LEARNING_RATE, R_LOSS_FACTOR)

BATCH_SIZE = 32
EPOCHS = 250
PRINT_EVERY_N_BATCHES = 100
INITIAL_EPOCH = 0

vae.train(
    x_train
    , batch_size = BATCH_SIZE
    , epochs = EPOCHS
    , run_folder = RUN_FOLDER
    , print_every_n_batches = PRINT_EVERY_N_BATCHES
    , initial_epoch = INITIAL_EPOCH
)
vae.save()

In [ ]:
LEARNING_RATE = 0.0000005
R_LOSS_FACTOR = 1000
vae.compile(LEARNING_RATE, R_LOSS_FACTOR)

BATCH_SIZE = 32
EPOCHS = 250
PRINT_EVERY_N_BATCHES = 100
INITIAL_EPOCH = 0

vae.train(
    x_train
    , batch_size = BATCH_SIZE
    , epochs = EPOCHS
    , run_folder = RUN_FOLDER
    , print_every_n_batches = PRINT_EVERY_N_BATCHES
    , initial_epoch = INITIAL_EPOCH
)
vae.save()

## Further Testing:

In [ ]:
#test again


vae = VariationalAutoencoder(
    input_dim = (32,32,3)
    , encoder_conv_filters = [32,64,64, 64]
    , encoder_conv_kernel_size = [3,3,3,3]
    , encoder_conv_strides = [1,2,2,1]
    , decoder_conv_t_filters = [64,64,32,1]
    , decoder_conv_t_kernel_size = [3,3,3,3]
    , decoder_conv_t_strides = [1,2,2,1]
    , z_dim = 2
)

#vae.load_weights("C:\\Users\\adoerr\\Desktop\\Machine Learning\\Aufgabe 2\\run\\weights\\weights-033-34.17.h5")


In [ ]:
testindex = 4999
img = x_train[testindex]
#print("Correct label =",label_name(y_test_numerical[testindex]))
print(plt.imshow(img))
img = img.reshape((1,) + img.shape)

encoding = vae.encoder.predict(img)
reconst = vae.decoder.predict(encoding)[0].squeeze()

print(encoding)

#filepath = os.path.join(self.run_folder, 'images','img_' + str(self.epoch).zfill(3) + '_' + str(batch) + '.jpg')
#print("Prediction:",vae.encoder.predict(img))
#print("Prediction:",vae.decoder.predict(vae.encoder.predict(img)))
#print(vae.decoder.predict(vae.encoder.predict(img)).type
#print(vae.encoder.predict)
#plt.imshow(vae.decoder.predict(vae.encoder.predict(img)))

In [ ]:
print(plt.imshow(reconst))

In [ ]:
import matplotlib.pyplot as plt
testindex = 51
img = x_test[testindex]
plt.imshow(img)
print(label_name(y_test_numerical[testindex]))